# Makes a big database for testing logistic regression fitting on the GPU

In [33]:
import os
import pandas as pd
import numpy as np
import scipy.stats as ss
from tqdm import tqdm

In [47]:
# let's make a 20 GB table, let's have 5 variables
# make some random number generators for each variable
x0_0 = ss.norm(loc=1.0, scale=0.1)
x0_1 = ss.norm(loc=0.5, scale=0.2)

x1_0 = ss.norm(loc=2.5, scale=1.5)
x1_1 = ss.norm(loc=-3.0, scale=1.5)

x2_0 = ss.norm(loc=10.0, scale=2.0)
x2_1 = ss.norm(loc=11.0, scale=2.1)

x3_0 = ss.norm(loc=4.5, scale=0.1)
x3_1 = ss.norm(loc=6.7, scale=0.2)

x4_0 = ss.norm(loc=-10, scale=3)
x4_1 = ss.norm(loc=-6, scale=2)

In [48]:
n_bytes_in_kilo = 1024
n_bytes_in_mega = n_bytes_in_kilo * 1024
n_bytes_in_giga = n_bytes_in_mega * 1024

n_bytes_total = 10.0 * n_bytes_in_giga

In [49]:
n_bytes_in_float = 4
n_bytes_per_row = 4 * 6
n_rows_in_database = int(n_bytes_total / n_bytes_per_row)

In [50]:
n_rows_in_database

447392426

In [51]:
# let's write to disk in sets of hundred thousand
n_rows_temp = int(1e5)
n_bytes_temp = n_rows_temp * n_bytes_per_row
n_bytes_temp

2400000

In [58]:
n_bytes_written = 0
n_gigs_written = n_bytes_written / n_bytes_in_giga
n_gigs_total = n_bytes_total / n_bytes_in_giga

df_file = os.path.join('..', 'data', 'test_db.hdf5')
if os.path.isfile(df_file):
    os.remove(df_file)

    
# bar = tqdm(total=n_gigs_total, desc='generating data and writing to file: {f}'.format(f=df_file))

while n_gigs_written < n_gigs_total:
    # generate data
    half_point = int(1e5) / 2
    temp_df = pd.DataFrame(index=range(int(1e5)), columns=['x0', 'x1', 'x2', 'x3', 'x4', 'y'], dtype=np.float32)
    temp_df.loc[:half_point-1, 'x0'] = x0_0.rvs(size=(half_point, )).astype(np.float32)
    temp_df.loc[half_point:, 'x0'] = x0_1.rvs(size=(half_point, )).astype(np.float32)
    temp_df.loc[:half_point-1, 'x1'] = x1_0.rvs(size=(half_point, )).astype(np.float32)
    temp_df.loc[half_point:, 'x1'] = x1_1.rvs(size=(half_point, )).astype(np.float32)
    temp_df.loc[:half_point-1, 'x2'] = x2_0.rvs(size=(half_point, )).astype(np.float32)
    temp_df.loc[half_point:, 'x2'] = x2_1.rvs(size=(half_point, )).astype(np.float32)
    temp_df.loc[:half_point-1, 'x3'] = x3_0.rvs(size=(half_point, )).astype(np.float32)
    temp_df.loc[half_point:, 'x3'] = x3_1.rvs(size=(half_point, )).astype(np.float32)
    temp_df.loc[:half_point-1, 'x4'] = x4_0.rvs(size=(half_point, )).astype(np.float32)
    temp_df.loc[half_point:, 'x4'] = x4_1.rvs(size=(half_point, )).astype(np.float32)
    temp_df.loc[:half_point-1, 'y'] = np.zeros(shape=(half_point, ), dtype=np.float32)
    temp_df.loc[half_point:, 'y'] = np.ones(shape=(half_point, ), dtype=np.float32)
    
    # shuffle
    temp_index_shuffled = np.random.permutation(temp_df.index)
    
    temp_df = temp_df.loc[temp_index_shuffled, :]
    
    # write it to the hdf file
    
    temp_df.to_hdf(df_file, 'table', mode='a', append=True)
    n_bytes_written += temp_df.values.nbytes
    n_gigs_written = n_bytes_written / n_bytes_in_giga
#     bar.update(n_gigs_written)
#     print(n_gigs_written)
    
    if n_bytes_written >= n_bytes_total:
        break
    
# bar.close()
    

/Users/antalek/anaconda/envs/ENVPy3/lib/python3.5/site-packages/scipy/stats/_continuous_distns.py:124: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return self._random_state.standard_normal(self._size)
/Users/antalek/anaconda/envs/ENVPy3/lib/python3.5/site-packages/ipykernel/__main__.py:26: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/antalek/anaconda/envs/ENVPy3/lib/python3.5/site-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)


In [59]:
bar.close()

In [60]:
n_gigs_written

10.000169277191162

In [61]:
n_gigs_total

10.0